In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --------------------------
# Load and Prepare the Data
# --------------------------
# Replace 'your_dataset.csv' with your actual dataset path
df = pd.read_csv('data.csv')

# Select 10 important columns and the target label
features = ['Machine', 'DebugSize', 'MajorImageVersion', 'ExportSize', 
            'IatVRA', 'NumberOfSections', 'SizeOfStackReserve', 
            'DllCharacteristics', 'ResourceSize', 'BitcoinAddresses']
target = 'Benign'

X = df[features].values
y = df[target].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape X for LSTM: [samples, sequence_length, features_per_step]
# Treat the 10 features as a sequence of length 10 with 1 feature per step
X = X.reshape(-1, 10, 1)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Split the data into training and validation sets (80/20 split)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --------------------------
# Define the LSTM Model
# --------------------------
class LSTMClassifier(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Initialize hidden state and cell state
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        # LSTM forward pass
        out, _ = self.lstm(x, (h0, c0))
        # Take the output from the last time step
        out = out[:, -1, :]
        out = self.fc(out)
        out = self.sigmoid(out)
        return out.squeeze()

# Instantiate the model
model = LSTMClassifier().to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --------------------------
# Train the Model
# --------------------------
print("\nTraining LSTM...")
model.train()
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    model.zero_grad()
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size].to(device)
        batch_y = y_train[i:i+batch_size].to(device)

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# --------------------------
# Evaluate the Model
# --------------------------
model.eval()
with torch.no_grad():
    X_val = X_val.to(device)
    y_val_pred = model(X_val)
    y_val_pred = (y_val_pred >= 0.5).float()
    y_val_pred = y_val_pred.cpu().numpy()
    y_val = y_val.cpu().numpy()

acc = accuracy_score(y_val, y_val_pred)
print(f"\nAccuracy for LSTM: {acc:.4f}")
print("Classification Report:")
print(classification_report(y_val, y_val_pred))

print("\n--------------------------------")
print(f"Model: LSTM with Accuracy: {acc:.4f}")
print("--------------------------------")

# Save the model and scaler
torch.save(model.state_dict(), 'lstm_model.pth')
joblib.dump(scaler, 'scaler.pkl')
print("Model saved to 'lstm_model.pth' and scaler saved to 'scaler.pkl'")

# --------------------------
# Prediction on Sample Data
# --------------------------
# Load the saved model and scaler
model = LSTMClassifier().to(device)
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()
scaler = joblib.load('scaler.pkl')
print("Loaded LSTM model from 'lstm_model.pth' and scaler from 'scaler.pkl'.")

# Provide a sample input as a dictionary (update these values as needed)
sample_data = {
    'Machine': 332,
    'DebugSize': 0,
    'MajorImageVersion': 0,
    'ExportSize': 0,
    'IatVRA': 2073,
    'NumberOfSections': 32768,
    'SizeOfStackReserve': 12,
    'DllCharacteristics': 1048576,
    'ResourceSize': 24044,
    'BitcoinAddresses': 0
}

# Create a DataFrame for the single sample
sample_df = pd.DataFrame([sample_data])
print("\nInput Data:")
print(sample_df)

# Prepare the sample for prediction
sample_X = sample_df[features].values
sample_X = scaler.transform(sample_X)
sample_X = sample_X.reshape(1, 10, 1)
sample_X = torch.tensor(sample_X, dtype=torch.float32).to(device)

# Make a prediction
with torch.no_grad():
    prediction = model(sample_X)
    prediction = (prediction >= 0.5).float().item()

# Get the probability (sigmoid output)
confidence = prediction if prediction >= 0.5 else 1 - prediction

# Map the prediction (1 = Benign, 0 = Malicious)
label_mapping = {1: "Benign", 0: "Malicious"}
predicted_label = label_mapping.get(int(prediction), "Unknown")

print("\nPrediction:")
print(f"Predicted Class: {predicted_label}")
print(f"Confidence: {confidence * 100:.2f}%")

Using device: cpu
